### Regularized (banded) CV regression workflow for Neuroscout
This notebook implements an encoding model for a single subject using Regularized Ridge Regression, as implemented in https://github.com/gallantlab/himalaya. In Neuroscout, this same pipeline should be run for all subjects.
- Input needed from the user
    - Define datasets (independent model fitting for all datasets)
    - Define cross-validation strategy
        - Across runs
        - Within runs
    - Define estimator
    - Define preprocessing steps (e.g., scaling?)
    - Define bands
    - Pass parameters
    - Output: scores, parameters, predicted time series
- Define outputs

<b> To do<b>:
- Decide which outputs to store
- Implement

In [137]:
import pyns
import pandas as pd
import nibabel as nib
import numpy as np
import glob
from copy import deepcopy
from pathlib import Path

In [138]:
api = pyns.Neuroscout()

In [144]:
dataset_name = 'Budapest'

### Choose subject

In [148]:
# Select subject from first run available in dataset
api.runs.get(dataset_name='Budapest')[0]

{'acquisition': None,
 'dataset_id': 27,
 'duration': 535.0,
 'id': 1435,
 'number': 3,
 'session': None,
 'subject': 'sid000005',
 'task': 48,
 'task_name': 'movie'}

In [149]:
subject = api.runs.get(dataset_name='Budapest')[0]['subject']

### Fetch predictors from Neuroscout and create design matrix
Let's retrieve predictor events for multiple sets of predictors. \
For now, let's pick three sets: <b>Audioset</b> + <b>MFCC</b> + <b>mel</b> features (plus some confounds).

In [150]:
audioset = ['as-Music',
            'as-Animal',
            'as-Whistling',
            'as-Vehicle',
            'as-Wild animals',
            'as-Thunderstorm',
            'as-Noise',
            'as-Fire',
            'as-Water',
            'as-Wind',
            'as-Glass',
            'as-Wood',
            'as-Silence',
            'as-Mechanisms',
            'as-Alarm',
            'as-Hands',
            'as-Tools',
            'as-Speech',
            'as-Explosion',
            'as-Engine',
            'as-Liquid',
            'as-Musical instrument']
mfccs = [f'mfcc_{i}' for i in range(20)]
mel = [f'mel_{i}' for i in range(64)]
confounds = ['rot_x', 'rot_y', 'rot_z', 'trans_x', 'trans_y', 'trans_z',
             'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02',
             'a_comp_cor_03','a_comp_cor_04','a_comp_cor_05']

Defining a function that does naive resampling of predictor events to TR.

Defining a function that takes a list of lists (`predictor_sets`), a dataset name, and a subject id and returns:
- `mats`, a list of pandas DataFrame (one per predictor set) of shape $n\_TRs x n\_features$;
- `run_index`, a list of the same length as `mats`, with run_ids for each row in `mats`.
Both `mats` and `run_index` are obtained by concatenating multiple runs. \
Each element of `predictor_sets` is a list of predictor names (e.g., `as-Speech`, `as-Music`). \
We also pass `resampling_ts` - not relevant for Neuroscout implementation, only needed for **ad hoc** resampling in this context.

In [151]:
from bids.variables import SparseRunVariable, BIDSRunVariableCollection
from bids.variables.entities import RunInfo
import math

TODO: Given a combined subject_df, group_by predictor / run, resample to TR, and combin

In [130]:
def fetch_predictors_as_collections(predictor_name, dataset_name, **entities):
    # Fetch from API
    all_df = api.predictor_events.get(
        predictor_name=predictor_name, dataset_name=dataset_name, output_type='df', **entities)
    all_df = all_df.rename(columns={'number': 'run', 'value': 'amplitude'})
    
    # Get run-level metadata
    all_run_info = {}
    for run_id in all_df.run_id.unique():
        resp = api.runs.get(run_id)
        ri = {
            'duration': resp['duration'],
            'tr': api.tasks.get(resp['task'])['TR'],
            'image': None
        }
        
        # TODO: Fetch real number of volumes, or allowing passing it in
        ri['n_vols'] = math.ceil(ri['duration'] / ri['tr'])
        all_run_info[run_id] = ri
        
    variables = []
    for (run_id, predictor_name), df in all_df.groupby(['run_id', 'predictor_name']):
        # Determine entities / run info
        keep_cols = []
        entities = {}
        for j in ['subject', 'session', 'run', 'acquisition', 'run_id']:
            val = df[j].iloc[0]
            if val:
                entities[j] = val
                keep_cols.append(j)
        run_info = RunInfo(**all_run_info[run_id], entities=entities)

        try:
            df['amplitude'] = pd.to_numeric(df['amplitude'])
        except ValueError:
            pass
        
        df = df[['onset', 'duration', 'amplitude'] + keep_cols].sort_values('onset')
        variables.append(SparseRunVariable(
            predictor_name, df, run_info, 'events'))
            
    return BIDSRunVariableCollection(variables=variables)

In [131]:
collection = fetch_predictors_as_collections(
    predictor_name=['as-Music', 'as-Speech'], dataset_name=dataset_name, subject=subject)

/home/alejandro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [277]:
def _make_input_matrices(collection, sort_by=['subject', 'run']):
    resampled = collection.to_dense().resample('TR')
    df = resampled.to_df()
    df = df.sort_values(sort_by)
    
    keep = list(collection.variables.keys())
    mat = df[keep]
    metadata = df.drop(columns=keep)
    
    return mat, metadata

In [288]:
X_vars, X_metadata = _make_input_matrices(collection)

### Fetch fMRI data and stack
Let's retrieve data for a couple of subjects from Budapest.
Neuroscout dataset can be found under the `neuroscout-datasets` organization: https://github.com/neuroscout-datasets

In [163]:
from datalad.api import install, get
from bids.layout import BIDSLayout

Install dataset and fetch relevant files

In [167]:
local_datasets = Path('/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/')
dataset_path = local_dataset_path / dataset_name
dataset_url = 'https://github.com/neuroscout-datasets/ds003017.git'

In [176]:
if not dataset_path.exists():
    install(path=local_dataset_path, source=dataset_url)
    
layout = BIDSLayout(dataset_path / 'fmriprep', derivatives=dataset_path / 'fmriprep', index_metadata=False)

/home/alejandro/anaconda3/lib/python3.7/site-packages/bids/layout/validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "
/home/alejandro/anaconda3/lib/python3.7/site-packages/bids/layout/validation.py:156: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [195]:
# Identify functional runs
subject_images = layout.get(subject=subject, desc='preproc', extension='.nii.gz', suffix='bold')
subject_images

[<BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-4_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-5

In [196]:
# get([f.path for f in subject_images, dataset=dataset_path)

In [261]:
def _stack_images(image_objects):
    arrays = []
    entities = []
    image_objects = sorted(image_objects, key=lambda x: x.entities['run'])
    for img in image_objects:
        data = np.asanyarray(nib.load(img.path).dataobj)
        run_y = data.reshape([data.shape[0] * data.shape[1] * data.shape[2], data.shape[3]]).T
        arrays.append(run_y)
        entities += [dict(img.entities)] * run_y.shape[0]
    entities = pd.DataFrame(entities)
    return np.vstack(arrays), entities

In [262]:
Y, img_idx = _stack_images(subject_images)

### Preprocessing and model fitting

Run the encoding model with lots of comments and printing.

In [136]:
from sklearn.model_selection import KFold, GroupKFold, PredefinedSplit
from himalaya.ridge import GroupRidgeCV
from himalaya.scoring import correlation_score

Cross-validated model fitting, prediction, and scoring loosely based on scikit-learn's [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). Returns a `results` dictionary with `'coefficients'`, `'test_predictions'`, and `'test_scores'` keys containing lists of numpy arrays for each outer cross-validation fold.

In [306]:
def _model_cv(estimator, X_vars, y, bands=None, groups=None,
              scoring=correlation_score, cv=None,
              inner_cv=None, confounds=None, split=None):
    # Container for results
    results = {
        'coefficients': [],
        'test_predictions': [],
        'test_scores': []}
    
    # If confounds, stack at the end
    if confounds is not None:
        bands.append(confounds)
    
    if bands is not None:
        X = []
        for band in bands:
            X.append(X_vars[band].as_matrix())
    else:
        X = X_vars.as_matrix()

    # Extract number of samples for convenience
    n_samples = y.shape[0]
    
    # Set default cross-validation to KFold if not specified
    cv = KFold() if not cv else cv
    
    # Loop through outer cross-validation folds
    for train, test in cv.split(np.arange(n_samples), groups=groups):
        
        # Get training model for list of model bands
        X_train = [x[train] for x in X] if type(X) == list else X[train]
        X_test = [x[test] for x in X] if type(X) == list else X[test]
        
        # Create inner cross-validation loop if specified
        if inner_cv:
            # Split inner cross-validation with groups if supplied
            inner_groups = np.array(groups)[train] if groups else groups
            inner_splits = inner_cv.split(np.arange(n_samples)[train],
                                          groups=inner_groups)
            
            # Update estimator with inner cross-validator
            estimator.set_params(cv=inner_splits)
            print(np.unique(inner_groups))
        
        # Fit the regression model on training data
        estimator.fit(X_train, y[train])
        
        # Zero out coefficients for confounds if provided
        if confounds is not None:
            estimator.coef_[-len(confounds):] = 0
        
        # Compute predictions with optional splitting by band
        test_prediction = estimator.predict(X_test, split=split)
        
        # Test scores should also optionally split by band
        test_score = scoring(y[test], test_prediction)
        
        # Populate results dictionary
        results['coefficients'].append(estimator.coef_)
        results['test_predictions'].append(test_prediction)
        results['test_scores'].append(test_score)
        
    return results

In [292]:
# Convert input data to numpy and fill NaNs
y = Y[:, :100]

# Default estimator should be GroupRidgeCV
estimator = GroupRidgeCV(groups='input')

# Default cross-validation should be leave-one-run-out
n_runs = len(idx['run_id'].unique())
cv = GroupKFold(n_splits=n_runs)
inner_cv = GroupKFold(n_splits=n_runs - 1)

In [21]:
# Run model with specified cross-validation, groups, confounds, and split outputs
results = _model_cv(estimator, X_vars, y, cv=cv, inner_cv=inner_cv, groups=X_metadata['run_id'].tolist(), split=True)

[1433 1434 1435 1436]
[........................................] 100% | 2.25 sec | 100 random sampling with cv | 
[1433 1434 1435 1437]
[........................................] 100% | 2.56 sec | 100 random sampling with cv | 
[1434 1435 1436 1437]
[........................................] 100% | 2.71 sec | 100 random sampling with cv | 
[1433 1434 1436 1437]
[........................................] 100% | 2.81 sec | 100 random sampling with cv | 
[1433 1435 1436 1437]
[........................................] 100% | 2.58 sec | 100 random sampling with cv | 


In [303]:
X_metadata['run_id'].tolist()

[1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,

In [22]:
results = _model_cv(estimator, X, y, cv=cv, inner_cv=inner_cv, groups=idx, confounds=confounds)

[1433 1434 1435 1436]
[........................................] 100% | 2.01 sec | 100 random sampling with cv | 
[1433 1434 1435 1437]
[........................................] 100% | 2.28 sec | 100 random sampling with cv | 
[1434 1435 1436 1437]
[........................................] 100% | 2.37 sec | 100 random sampling with cv | 
[1433 1434 1436 1437]
[........................................] 100% | 2.49 sec | 100 random sampling with cv | 
[1433 1435 1436 1437]
[........................................] 100% | 2.77 sec | 100 random sampling with cv | 


In [23]:
results = _model_cv(estimator, X, y, cv=KFold(), inner_cv=KFold())

[None]
[........................................] 100% | 2.57 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.67 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.38 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.73 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.68 sec | 100 random sampling with cv | 


In [127]:
results = _model_cv(estimator, X, y, cv=KFold())

[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[..................................      ] 86% | 0.10 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.12 sec | 100 random sampling with cv | 
[........................................] 100% | 0.11 sec | 100 random sampling with cv | 
[..................................      ] 85% | 0.09 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[........................................] 100% | 0.10 sec | 100 random sampling with cv | 


/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [128]:
results = _model_cv(estimator, X, y, inner_cv=KFold())

[None]
[........................................] 100% | 2.77 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.67 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.78 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.69 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.65 sec | 100 random sampling with cv | 


In [117]:
results = _model_cv(estimator, X, y, groups=idx)

[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[...................................     ] 89% | 0.09 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[........................................] 100% | 0.12 sec | 100 random sampling with cv | 
[............................            ] 71% | 0.08 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.11 sec | 100 random sampling with cv | 
[........................................] 100% | 0.11 sec | 100 random sampling with cv | 


/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [307]:
# Using single-band (non-banded) model with sklearn RidgeCV
from sklearn.linear_model import RidgeCV
results = _model_cv(RidgeCV(), X_vars, y, cv=cv, inner_cv=inner_cv, groups=X_metadata['run_id'].tolist())

/home/alejandro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[1433 1434 1435 1436]


TypeError: predict() got an unexpected keyword argument 'split'

In [ ]:
me

### Handling outputs

In [ ]:
### 

### Validate against other workflows

In [ ]:
### 